In [1]:
import pandas as pd
import torch 

df=pd.read_csv("train_clean_kalman.csv")
df['signal']=df['signal']/df['signal'].max()

In [2]:
df['open_channels'].max()

1

# Create time series dataset

In [3]:
class TimesereisDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.Series(), seqLen=10):
        super().__init__()
        self.df=df
        self.seqLen=seqLen
    
    def __len__(self):
        return self.df.shape[0]-self.seqLen-1
    
    def __getitem__(self,index):
        x=self.df.iloc[index:index+self.seqLen,1].values
        y=self.df.iloc[index+self.seqLen-1,2] 
        return x,y   
seqLen=50
timeSeriesDataset = TimesereisDataset(df,seqLen)

In [4]:
trainNumbers=int(len(timeSeriesDataset)*0.9)
trainDataset,testDataset=torch.utils.data.random_split(timeSeriesDataset,[trainNumbers,len(timeSeriesDataset)-trainNumbers])
trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=8,shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=8,shuffle=True)

In [5]:

from Wavenet import WaveNet,WaveNetClassifier
from tqdm import tqdm 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
min_classes = 2  # or whatever minimum number of classes you expect
output_size = max(int(df['open_channels'].max()) + 1, min_classes)
wavenetClassifierModel = WaveNetClassifier(seqLen, output_size)
# prv: wavenetClassifierModel=WaveNetClassifier(seqLen,df['open_channels'].max()+1)
wavenetClassifierModel.to(device)

receptive_field = wavenetClassifierModel.wavenet.calculateReceptiveField()
print(f"Sequence Length: {seqLen}")
print(f"Receptive Field: {receptive_field}")
print(f"Linear layer input size: {seqLen - receptive_field}")
print(f"Output size: {output_size}")


wavenetClassifierModel.train()

optimizer=torch.optim.AdamW(wavenetClassifierModel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossFunction = torch.nn.CrossEntropyLoss()

def calc_accuracy(Out,Y):
    max_vals, max_indices = torch.max(Out,1)
    train_acc = (max_indices == Y).sum().item()/max_indices.size()[0]
    return train_acc
  
epochs=1
globalStep=500

for epoch in range(epochs):
    for step, (x_train,y_train) in tqdm(enumerate(trainDataLoader),desc="Training"):
         # First reshape and convert to float
         x_train = torch.unsqueeze(x_train,dim=1).float()
         # Then move to device
         x_train = x_train.to(device)
         y_train = y_train.to(device)
         # Then pass to model
         output = wavenetClassifierModel(x_train)
         output = torch.squeeze(output,dim=1)

         loss= lossFunction(output,y_train)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         if step%globalStep==0:
            # scheduler.step()
            # print(output.detach().numpy())
            # print(y_train.numpy())
            with torch.no_grad():
                accuracy=0
                loss=0
                for stepTest, (x_test,y_test) in tqdm(enumerate(testDataLoader),desc="Validation"):
                    # Same order for test data
                    x_test = torch.unsqueeze(x_test,dim=1).float()
                    x_test = x_test.to(device)
                    y_test = y_test.to(device)
                    output = wavenetClassifierModel(x_test)

                    output = torch.squeeze(output,dim=1)
                    accuracy+=calc_accuracy(output,y_test)*100
                    loss+= lossFunction(output,y_test).item()
                    if stepTest>200:
                        break
            print(f"loss for step {step} : {loss/stepTest}  :  Accuracy: {accuracy/stepTest} %")

         
    print(f"epch {epoch}")
         

WaveNet config:
Input sequence length: 50
Receptive field: 45
Expected output length: 5
Linear layer input size: 4
Linear layer output size: 2
Sequence Length: 50
Receptive Field: 45
Linear layer input size: 5
Output size: 2


Training: 0it [00:00, ?it/s]/Users/jos/miniforge3/envs/env_pytorch_312/lib/python3.12/site-packages/torch/nn/modules/conv.py:370: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1037.)
  return F.conv1d(


Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])


Validation: 12it [00:00, 307.81it/s]
Training: 6it [00:00, 56.56it/s]

Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet sh

...

After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After fla

Training: 78it [00:00, 99.06it/s] 

After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After fla

Training: 107it [00:01, 106.07it/s]

Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet shape: torch.Size([8, 1, 4])
After flatten shape: torch.Size([8, 4])
After linear shape: torch.Size([8, 2])
Input shape: torch.Size([8, 1, 50])
After WaveNet sh